# Data Analysis - Climate Data



In [1]:
#Importing Necessary Packages

import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import statsmodels.api as sm 
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
from statsmodels.miscmodels.ordinal_model import OrderedModel
from pystout import pystout
from tabulate import tabulate
import latextable

In [2]:
def confint(data):
    ci = 1.96 * np.std(data) / np.sqrt(len(data)-1)
    return ci

In [3]:
spei_data = pd.DataFrame()

spei_data = pd.read_csv('Data/SPEI_27.75_84.25.csv')

spei03 = spei_data[['DATA', 'SPEI_3']]
start_date = 759 #Index for April 2013, first month of Nepali Year 2070 (2 years before calendar data)
timehorizon = 9 * 12 #End of Nepali year 2078 is April 2022
end_date = start_date + timehorizon

spei03_time = spei03[start_date:end_date]

spei03_time

,DATA,SPEI_3
759,Apr2013,0.81887
760,May2013,1.15322
761,Jun2013,1.56373
762,Jul2013,0.31788
763,Aug2013,-1.16400
...,...,...
862,Nov2021,-0.63905
863,Dec2021,1.09034
864,Jan2022,0.67670
865,Feb2022,1.57778


In [4]:
#Calculating average SPEI for rice and maize growing seasons, according to FAO Crop Calendar for Nepal Terai region: https://cropcalendar.apps.fao.org/#/home?id=NP&crops=0338,0113

rice_sowing = 1 #First instance of May in spei03_time
rice_harvest = 5 #Index for first instance of Sept

maize_sowing = 7 #Index for Nov.
maize_harvest = 12 #Index for March

years = np.arange(0, 9, 1)

rice_spei = np.zeros(len(years))
maize_spei = np.zeros(len(years))

for y in years:
    rice_spei[y] = np.average(spei03_time['SPEI_3'][rice_sowing + 12*y:rice_harvest + 12*y])
    maize_spei[y] = np.average(spei03_time['SPEI_3'][maize_sowing + 12*y:maize_harvest + 12*y])                         
                        
print(rice_spei)
print(maize_spei)

[ 0.4677075 -1.29173   -1.3790125 -1.17149   -0.74029   -0.7550525
 -1.6936725  0.79596    1.3974825]
[ 0.3031    0.76527  -1.86876  -0.023868 -1.510392 -0.380582  0.912888
 -1.270394  0.573058]


In [5]:
#Creating dataframe with Nepali Year and SPEI values
data = {'Year': [], 'RiceSPEI':[], 'MaizeSPEI': [], 'RiceSPEI_1yearlag':[], 'MaizeSPEI_1yearlag': [], 'RiceSPEI_2yearlag':[], 'MaizeSPEI_2yearlag':[]}
years = [72, 73, 74, 75, 76, 77, 78]

for i, y in enumerate(years):
    data['Year'].append(y)
    data['RiceSPEI'].append(rice_spei[i+2])
    data['MaizeSPEI'].append(maize_spei[i+2])
    data['MaizeSPEI_1yearlag'].append(rice_spei[i+1])
    data['RiceSPEI_1yearlag'].append(maize_spei[i+1])
    data['RiceSPEI_2yearlag'].append(rice_spei[i])
    data['MaizeSPEI_2yearlag'].append(maize_spei[i])
    

climate_df = pd.DataFrame(data)
climate_df

,Year,RiceSPEI,MaizeSPEI,RiceSPEI_1yearlag,MaizeSPEI_1yearlag,RiceSPEI_2yearlag,MaizeSPEI_2yearlag
0,72,-1.379012,-1.868760,0.765270,-1.291730,0.467707,0.303100
1,73,-1.171490,-0.023868,-1.868760,-1.379012,-1.291730,0.765270
2,74,-0.740290,-1.510392,-0.023868,-1.171490,-1.379012,-1.868760
3,75,-0.755053,-0.380582,-1.510392,-0.740290,-1.171490,-0.023868
4,76,-1.693672,0.912888,-0.380582,-0.755053,-0.740290,-1.510392
5,77,0.795960,-1.270394,0.912888,-1.693672,-0.755053,-0.380582
6,78,1.397482,0.573058,-1.270394,0.795960,-1.693672,0.912888


In [7]:
#Converting SPEI indices to dummy variables for drought
dummies = ['RiceDrought', 'MaizeDrought', 'RiceDrought_1yearlag', 'MaizeDrought_1yearlag', 'RiceDrought_2yearlag', 'MaizeDrought_2yearlag']
drought_threshold = -1
for d, col in zip(dummies, climate_df.columns[1:]):
    climate_df[d] = np.where(climate_df[col] < drought_threshold, 1, 0)

climate_df.to_csv('ClimateData.csv')
climate_df



,Year,RiceSPEI,MaizeSPEI,RiceSPEI_1yearlag,MaizeSPEI_1yearlag,RiceSPEI_2yearlag,MaizeSPEI_2yearlag,RiceDrought,MaizeDrought,RiceDrought_1yearlag,MaizeDrought_1yearlag,RiceDrought_2yearlag,MaizeDrought_2yearlag
0,72,-1.379012,-1.868760,0.765270,-1.291730,0.467707,0.303100,1,1,0,1,0,0
1,73,-1.171490,-0.023868,-1.868760,-1.379012,-1.291730,0.765270,1,0,1,1,1,0
2,74,-0.740290,-1.510392,-0.023868,-1.171490,-1.379012,-1.868760,0,1,0,1,1,1
3,75,-0.755053,-0.380582,-1.510392,-0.740290,-1.171490,-0.023868,0,0,1,0,1,0
4,76,-1.693672,0.912888,-0.380582,-0.755053,-0.740290,-1.510392,1,0,0,0,0,1
5,77,0.795960,-1.270394,0.912888,-1.693672,-0.755053,-0.380582,0,1,0,1,0,0
6,78,1.397482,0.573058,-1.270394,0.795960,-1.693672,0.912888,0,0,1,0,1,0
